In [1]:
# 1. CARREGAMENTO DOS MODELOS TREINADOS
from utils import *
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import os
import pickle
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações
config = carregar_configuracoes()
print("=== CARREGANDO MODELOS TREINADOS ===")

# Carregar modelo de classificação de espécies
print("📂 Carregando modelo de espécies...")
modelo_especies = load_model('modelos_salvos/melhor_modelo_especies_final_otimizado.h5')
with open('datasets_processados/label_encoder_especies_modelo.pkl', 'rb') as f:
    encoder_especies = pickle.load(f)

print(f"   ✅ Modelo de espécies: {encoder_especies.classes_}")

# Carregar modelos de classificação de saúde
print("📂 Carregando modelos especialistas...")
modelos_especialistas = {}
especies_disponiveis = []

for especie in ['tomato', 'potato', 'pepper']:
    modelo_path = f'modelos_salvos/especialistas/especialista_{especie}_binario_final.h5'
    if os.path.exists(modelo_path):
        modelos_especialistas[especie] = load_model(modelo_path)
        especies_disponiveis.append(especie)
        print(f"   ✅ Modelo {especie}: carregado")
    else:
        print(f"   ⚠️ Modelo {especie}: não encontrado")


2025-07-06 23:18:14.565796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751854694.630642  264921 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751854694.652611  264921 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751854694.757982  264921 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751854694.758021  264921 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751854694.758023  264921 computation_placer.cc:177] computation placer alr

=== CARREGANDO MODELOS TREINADOS ===
📂 Carregando modelo de espécies...


I0000 00:00:1751854699.325984  264921 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4047 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5
/home/gustavo/.local/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


   ✅ Modelo de espécies: ['Pepper_bell' 'Potato' 'Tomato']
📂 Carregando modelos especialistas...


   ✅ Modelo tomato: carregado


   ✅ Modelo potato: carregado


   ✅ Modelo pepper: carregado


In [2]:
# 2. IMPLEMENTAÇÃO DO PIPELINE HIERÁRQUICO
def preprocessar_imagem(caminho_imagem, target_size=(224, 224)):
    """Preprocessa imagem para os modelos"""
    img = image.load_img(caminho_imagem, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def pipeline_hierarquico(caminho_imagem, modelo_especies, encoder_especies, modelos_especialistas, threshold_confianca=0.5):
    """
    Pipeline completo: Passo 3 (espécie) → Passo 4 (saúde)
    
    Returns:
        dict: {
            'especie_predita': str,
            'confianca_especie': float,
            'saude_predita': str,
            'confianca_saude': float,
            'resultado_final': str,
            'confianca_final': float,
            'etapas': dict
        }
    """
    # Preprocessar imagem
    img_preprocessada = preprocessar_imagem(caminho_imagem)
    
    #Classificar espécie
    pred_especies = modelo_especies.predict(img_preprocessada, verbose=0)
    indice_especie = np.argmax(pred_especies)
    especie_predita = encoder_especies.inverse_transform([indice_especie])[0]
    confianca_especie = np.max(pred_especies)
    
    # Mapear nome da espécie para o modelo especialista
    mapeamento_especies = {
        'Tomato': 'tomato',
        'Potato': 'potato', 
        'Pepper_bell': 'pepper'
    }
    
    especie_modelo = mapeamento_especies.get(especie_predita)
    
    #Classificar saúde
    if especie_modelo and especie_modelo in modelos_especialistas:
        modelo_especialista = modelos_especialistas[especie_modelo]
        pred_saude = modelo_especialista.predict(img_preprocessada, verbose=0)[0][0]
        
        # Conversão binária: >0.5 = unhealthy, <=0.5 = healthy
        if pred_saude > 0.5:
            saude_predita = 'unhealthy'
            confianca_saude = pred_saude
        else:
            saude_predita = 'healthy'
            confianca_saude = 1 - pred_saude
            
        # Resultado final combinado
        resultado_final = f"{especie_predita}_{saude_predita}"
        confianca_final = confianca_especie * confianca_saude  # Propagação de incerteza
        
        sucesso_pipeline = True
        
    else:
        # Modelo especialista não disponível
        saude_predita = 'unknown'
        confianca_saude = 0.0
        resultado_final = f"{especie_predita}_unknown"
        confianca_final = confianca_especie
        sucesso_pipeline = False
    
    return {
        'especie_predita': especie_predita,
        'confianca_especie': float(confianca_especie),
        'saude_predita': saude_predita,
        'confianca_saude': float(confianca_saude),
        'resultado_final': resultado_final,
        'confianca_final': float(confianca_final),
        'sucesso_pipeline': sucesso_pipeline,
        'etapas': {
            'passo3_especies': {'pred': especie_predita, 'conf': float(confianca_especie)},
            'passo4_saude': {'pred': saude_predita, 'conf': float(confianca_saude)}
        }
    }

print("✅ Pipeline hierárquico implementado!")


# Encontrar uma imagem de teste
test_image = None
for especie in ['Tomato', 'Potato', 'Pepper_bell']:
    for classe in config['especialistas'][especie]['classes']:
        pasta_classe = os.path.join(config['base_path'], classe)
        if os.path.exists(pasta_classe):
            images = [f for f in os.listdir(pasta_classe) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            if images:
                test_image = os.path.join(pasta_classe, images[0])
                test_class = classe
                break
    if test_image:
        break

if test_image:
    print(f"\n🧪 Testando com: {test_class}")
    resultado = pipeline_hierarquico(test_image, modelo_especies, encoder_especies, modelos_especialistas)
    print(f"   Espécie: {resultado['especie_predita']} (conf: {resultado['confianca_especie']:.3f})")
    print(f"   Saúde: {resultado['saude_predita']} (conf: {resultado['confianca_saude']:.3f})")
    print(f"   Final: {resultado['resultado_final']} (conf: {resultado['confianca_final']:.3f})")
else:
    print("⚠️ Nenhuma imagem de teste encontrada")


✅ Pipeline hierárquico implementado!

🧪 Testando com: Tomato_Bacterial_spot


I0000 00:00:1751854742.309154  265008 service.cc:152] XLA service 0x7513c80029d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751854742.311132  265008 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2025-07-06 23:19:03.302621: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751854744.992196  265008 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-07-06 23:19:10.227061: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.172 = (f32[1,128,28,28]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,128,28,28]{3,2,1,0} %bitcast.5016, f32[128,128,3,3]{3,2,1,0} %bitcast.5023, f32[128]{0} %bitcast.5025), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_targ

   Espécie: Tomato (conf: 0.665)
   Saúde: unhealthy (conf: 1.000)
   Final: Tomato_unhealthy (conf: 0.665)


In [3]:
# 3. CRIAÇÃO DO CONJUNTO DE TESTE HIERÁRQUICO
def criar_conjunto_teste_hierarquico(config, n_samples_per_class=50):
    """
    Cria conjunto de teste balanceado para avaliação hierárquica
    """
    
    conjunto_teste = []
    
    for especie, info in config['especialistas'].items():
        print(f"📂 Processando {especie}...")
        
        for classe in info['classes']:
            pasta_classe = os.path.join(config['base_path'], classe)
            
            if not os.path.exists(pasta_classe):
                print(f"   ⚠️ Pasta não encontrada: {classe}")
                continue
                
            # Obter imagens da classe
            images = [f for f in os.listdir(pasta_classe) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            if len(images) == 0:
                print(f"   ⚠️ Nenhuma imagem em: {classe}")
                continue
                
            # Selecionar amostra aleatória
            np.random.seed(42)
            n_select = min(n_samples_per_class, len(images))
            selected_images = np.random.choice(images, n_select, replace=False)
            
            # Determinar rótulos
            especie_label = especie
            saude_label = 'healthy' if 'healthy' in classe.lower() else 'unhealthy'
            combined_label = f"{especie}_{saude_label}"
            
            # Adicionar ao conjunto
            for img_name in selected_images:
                caminho_completo = os.path.join(pasta_classe, img_name)
                conjunto_teste.append({
                    'caminho': caminho_completo,
                    'classe_original': classe,
                    'especie_real': especie_label,
                    'saude_real': saude_label,
                    'combined_real': combined_label
                })
            
            print(f"   ✅ {classe}: {n_select} imagens selecionadas")
    
    # Converter para DataFrame
    df_teste = pd.DataFrame(conjunto_teste)
    
    print(f"\n📊 CONJUNTO DE TESTE CRIADO:")
    print(f"   Total de imagens: {len(df_teste)}")
    print(f"   Espécies: {df_teste['especie_real'].value_counts().to_dict()}")
    print(f"   Saúde: {df_teste['saude_real'].value_counts().to_dict()}")
    
    return df_teste

# Criar conjunto de teste
df_teste_hierarquico = criar_conjunto_teste_hierarquico(config, n_samples_per_class=30)

# Salvar para reutilização
df_teste_hierarquico.to_csv('datasets_processados/conjunto_teste_hierarquico.csv', index=False)
print("\n💾 Conjunto de teste salvo em: datasets_processados/conjunto_teste_hierarquico.csv")


📂 Processando Tomato...
   ✅ Tomato_Bacterial_spot: 30 imagens selecionadas
   ✅ Tomato_Early_blight: 30 imagens selecionadas
   ✅ Tomato_Late_blight: 30 imagens selecionadas
   ✅ Tomato_Leaf_Mold: 30 imagens selecionadas
   ✅ Tomato_Septoria_leaf_spot: 30 imagens selecionadas
   ✅ Tomato_Spider_mites_Two_spotted_spider_mite: 30 imagens selecionadas
   ✅ Tomato__Target_Spot: 30 imagens selecionadas
   ✅ Tomato__Tomato_YellowLeaf__Curl_Virus: 30 imagens selecionadas
   ✅ Tomato__Tomato_mosaic_virus: 30 imagens selecionadas
   ✅ Tomato_healthy: 30 imagens selecionadas
📂 Processando Potato...
   ✅ Potato___Early_blight: 30 imagens selecionadas
   ✅ Potato___Late_blight: 30 imagens selecionadas
   ✅ Potato___healthy: 30 imagens selecionadas
📂 Processando Pepper_bell...
   ✅ Pepper__bell___Bacterial_spot: 30 imagens selecionadas
   ✅ Pepper__bell___healthy: 30 imagens selecionadas

📊 CONJUNTO DE TESTE CRIADO:
   Total de imagens: 450
   Espécies: {'Tomato': 300, 'Potato': 90, 'Pepper_bell':

In [4]:
# 4. AVALIAÇÃO COMPLETA DO PIPELINE HIERÁRQUICO
def avaliar_pipeline_hierarquico(df_teste, modelo_especies, encoder_especies, modelos_especialistas):
    """
    Avalia o pipeline hierárquico completo
    """
    
    resultados = []
    
    # Processar cada imagem
    for idx, row in df_teste.iterrows():
        if idx % 50 == 0:
            print(f"   Processando {idx+1}/{len(df_teste)}...")
            
        # Aplicar pipeline hierárquico
        resultado = pipeline_hierarquico(
            row['caminho'], 
            modelo_especies, 
            encoder_especies, 
            modelos_especialistas
        )
        
        # Adicionar dados reais
        resultado['especie_real'] = row['especie_real']
        resultado['saude_real'] = row['saude_real']
        resultado['combined_real'] = row['combined_real']
        resultado['classe_original'] = row['classe_original']
        
        # Calcular acertos
        resultado['acerto_especie'] = resultado['especie_predita'] == resultado['especie_real']
        resultado['acerto_saude'] = resultado['saude_predita'] == resultado['saude_real']
        resultado['acerto_combined'] = resultado['resultado_final'] == resultado['combined_real']
        
        resultados.append(resultado)
    
    # Converter para DataFrame
    df_resultados = pd.DataFrame(resultados)
    
    return df_resultados

# Executar avaliação
print("🚀 Iniciando avaliação completa...")
df_resultados = avaliar_pipeline_hierarquico(
    df_teste_hierarquico, 
    modelo_especies, 
    encoder_especies, 
    modelos_especialistas
)

# Salvar resultados
df_resultados.to_csv('datasets_processados/resultados_pipeline_hierarquico.csv', index=False)
print("\n💾 Resultados salvos em: datasets_processados/resultados_pipeline_hierarquico.csv")

print(f"\n✅ Avaliação concluída: {len(df_resultados)} predições realizadas")


🚀 Iniciando avaliação completa...
   Processando 1/450...
   Processando 51/450...
   Processando 101/450...
   Processando 151/450...
   Processando 201/450...
   Processando 251/450...
   Processando 301/450...
   Processando 351/450...
   Processando 401/450...

💾 Resultados salvos em: datasets_processados/resultados_pipeline_hierarquico.csv

✅ Avaliação concluída: 450 predições realizadas


In [5]:
# 5. ANÁLISE DE PROPAGAÇÃO DE ERROS E MÉTRICAS
def analisar_propagacao_erros(df_resultados):
    """
    Analisa como erros se propagam no pipeline hierárquico
    """
    
    # Métricas por etapa
    acc_especie = df_resultados['acerto_especie'].mean()
    acc_saude = df_resultados['acerto_saude'].mean()
    acc_combined = df_resultados['acerto_combined'].mean()
    
    print("📊 ACURÁCIAS POR ETAPA:")
    print(f"Classificação de espécie: {acc_especie:.4f} ({acc_especie*100:.1f}%)")
    print(f"Classificação de saúde: {acc_saude:.4f} ({acc_saude*100:.1f}%)")
    print(f"Pipeline completo: {acc_combined:.4f} ({acc_combined*100:.1f}%)")
    
    # Análise de propagação de erros
    print(f"\n🔍 ANÁLISE DE PROPAGAÇÃO:")
    
    # Casos onde espécie está correta
    correto_especie = df_resultados[df_resultados['acerto_especie'] == True]
    if len(correto_especie) > 0:
        acc_saude_dado_especie_correta = correto_especie['acerto_saude'].mean()
        print(f"   Acurácia de saúde QUANDO espécie está correta: {acc_saude_dado_especie_correta:.4f} ({acc_saude_dado_especie_correta*100:.1f}%)")
    
    # Casos onde espécie está incorreta
    incorreto_especie = df_resultados[df_resultados['acerto_especie'] == False]
    if len(incorreto_especie) > 0:
        acc_saude_dado_especie_incorreta = incorreto_especie['acerto_saude'].mean()
        print(f"   Acurácia de saúde QUANDO espécie está incorreta: {acc_saude_dado_especie_incorreta:.4f} ({acc_saude_dado_especie_incorreta*100:.1f}%)")
    
    # Impacto da propagação
    acc_teorica = acc_especie * acc_saude_dado_especie_correta if len(correto_especie) > 0 else 0
    impacto_propagacao = acc_combined - acc_teorica
    
    print(f"\n⚖️ IMPACTO DA PROPAGAÇÃO:")
    print(f"   Acurácia teórica (independente): {acc_teorica:.4f}")
    print(f"   Acurácia real (hierárquica): {acc_combined:.4f}")
    print(f"   Impacto da propagação: {impacto_propagacao:+.4f}")
    
    # Matriz de confusão por etapa
    print(f"\n📈 MATRIZES DE CONFUSÃO:")
    
    # Espécie
    print(f"\n🌱 CLASSIFICAÇÃO DE ESPÉCIE:")
    especies_reais = df_resultados['especie_real'].values
    especies_pred = df_resultados['especie_predita'].values
    print(classification_report(especies_reais, especies_pred, zero_division=0))
    
    # Saúde
    print(f"\n🏥 CLASSIFICAÇÃO DE SAÚDE:")
    saude_reais = df_resultados['saude_real'].values
    saude_pred = df_resultados['saude_predita'].values
    print(classification_report(saude_reais, saude_pred, zero_division=0))
    
    # Análise por espécie
    print(f"\n🔬 ANÁLISE POR ESPÉCIE:")
    for especie in df_resultados['especie_real'].unique():
        subset = df_resultados[df_resultados['especie_real'] == especie]
        acc_esp = subset['acerto_especie'].mean()
        acc_sau = subset['acerto_saude'].mean()
        acc_comb = subset['acerto_combined'].mean()
        
        print(f"   {especie}:")
        print(f"     Espécie: {acc_esp:.3f} | Saúde: {acc_sau:.3f} | Completo: {acc_comb:.3f}")
    
    # Confiança média
    print(f"\n🎯 ANÁLISE DE CONFIANÇA:")
    conf_especie_media = df_resultados['confianca_especie'].mean()
    conf_saude_media = df_resultados['confianca_saude'].mean()
    conf_final_media = df_resultados['confianca_final'].mean()
    
    print(f"   Confiança média - Espécie: {conf_especie_media:.3f}")
    print(f"   Confiança média - Saúde: {conf_saude_media:.3f}")
    print(f"   Confiança média - Final: {conf_final_media:.3f}")
    
    return {
        'acc_especie': acc_especie,
        'acc_saude': acc_saude,
        'acc_combined': acc_combined,
        'acc_teorica': acc_teorica,
        'impacto_propagacao': impacto_propagacao,
        'conf_especie_media': conf_especie_media,
        'conf_saude_media': conf_saude_media,
        'conf_final_media': conf_final_media
    }

# Executar análise
metricas = analisar_propagacao_erros(df_resultados)


📊 ACURÁCIAS POR ETAPA:
Classificação de espécie: 0.8867 (88.7%)
Classificação de saúde: 0.8667 (86.7%)
Pipeline completo: 0.7689 (76.9%)

🔍 ANÁLISE DE PROPAGAÇÃO:
   Acurácia de saúde QUANDO espécie está correta: 0.8672 (86.7%)
   Acurácia de saúde QUANDO espécie está incorreta: 0.8627 (86.3%)

⚖️ IMPACTO DA PROPAGAÇÃO:
   Acurácia teórica (independente): 0.7689
   Acurácia real (hierárquica): 0.7689
   Impacto da propagação: +0.0000

📈 MATRIZES DE CONFUSÃO:

🌱 CLASSIFICAÇÃO DE ESPÉCIE:
              precision    recall  f1-score   support

 Pepper_bell       0.81      0.92      0.86        60
      Potato       0.71      0.93      0.80        90
      Tomato       0.99      0.87      0.92       300

    accuracy                           0.89       450
   macro avg       0.83      0.91      0.86       450
weighted avg       0.91      0.89      0.89       450


🏥 CLASSIFICAÇÃO DE SAÚDE:
              precision    recall  f1-score   support

     healthy       0.75      0.50      0.60  

In [6]:
# 6. IMPLEMENTAÇÃO DE BASELINE PARA COMPARAÇÃO
def calcular_baseline_direto(df_resultados):
    """
    Calcula acurácia baseline mais realista simulando um classificador direto
    que usa apenas as predições de espécie (sem hierarquia)
    """
    
    # BASELINE REALISTA: Usar apenas as predições de espécie + assumir saúde majoritária
    # Isso simula um classificador que só identifica espécie e "chuta" a saúde
    
    print("📊 BASELINE DIRETO (Classificação baseada apenas em espécie):")
    print("   Método: Predição de espécie + classificação majoritária de saúde")
    
    # Calcular a classe majoritária de saúde por espécie no conjunto de dados
    classes_majoritarias = {}
    for especie in df_resultados['especie_real'].unique():
        subset = df_resultados[df_resultados['especie_real'] == especie]
        classe_majoritaria = subset['saude_real'].mode()[0]  # Classe mais comum
        classes_majoritarias[especie] = classe_majoritaria
        
        print(f"   {especie}: classe majoritária = {classe_majoritaria}")
    
    # Simular baseline: usar predição de espécie + classe majoritária de saúde
    acertos_baseline = 0
    total_amostras = len(df_resultados)
    
    for idx, row in df_resultados.iterrows():
        # Usar a predição de espécie do pipeline hierárquico
        especie_pred = row['especie_predita']
        
        # Usar classe majoritária de saúde para essa espécie
        saude_baseline = classes_majoritarias.get(especie_pred, 'unhealthy')
        
        # Construir resultado baseline
        resultado_baseline = f"{especie_pred}_{saude_baseline}"
        
        # Verificar se acertou
        if resultado_baseline == row['combined_real']:
            acertos_baseline += 1
    
    acc_baseline = acertos_baseline / total_amostras
    
    print(f"   Acertos baseline: {acertos_baseline}/{total_amostras}")
    print(f"   Acurácia baseline: {acc_baseline:.4f} ({acc_baseline*100:.1f}%)")
    
    return acc_baseline

def calcular_baseline_teorico(df_resultados):
    """
    Calcula o baseline teórico baseado na multiplicação das acurácias independentes
    """
    
    acc_especie = df_resultados['acerto_especie'].mean()
    acc_saude = df_resultados['acerto_saude'].mean()
    acc_teorico = acc_especie * acc_saude
    
    print(f"\n📊 BASELINE TEÓRICO (Multiplicação de acurácias independentes):")
    print(f"   Acurácia espécie: {acc_especie:.4f} ({acc_especie*100:.1f}%)")
    print(f"   Acurácia saúde: {acc_saude:.4f} ({acc_saude*100:.1f}%)")
    print(f"   Acurácia teórica: {acc_teorico:.4f} ({acc_teorico*100:.1f}%)")
    
    return acc_teorico

# Calcular baselines
print("🔍 Calculando baselines para comparação...")
acc_baseline_direto = calcular_baseline_direto(df_resultados)
acc_baseline_teorico = calcular_baseline_teorico(df_resultados)

# 7. COMPARAÇÃO FINAL E CONCLUSÕES
def comparacao_final(metricas, acc_baseline_direto, acc_baseline_teorico):
    """
    Compara pipeline hierárquico com diferentes baselines
    """
    
    acc_hierarquico = metricas['acc_combined']
    diferenca_direto = acc_hierarquico - acc_baseline_direto
    diferenca_teorico = acc_hierarquico - acc_baseline_teorico
    
    print("\n📊 RESULTADOS FINAIS:")
    print(f"   🔗 Pipeline Hierárquico: {acc_hierarquico:.4f} ({acc_hierarquico*100:.1f}%)")
    print(f"   📍 Baseline Direto: {acc_baseline_direto:.4f} ({acc_baseline_direto*100:.1f}%)")
    print(f"   📐 Baseline Teórico: {acc_baseline_teorico:.4f} ({acc_baseline_teorico*100:.1f}%)")
    
    print(f"\n📈 COMPARAÇÕES:")
    print(f"   vs Baseline Direto: {diferenca_direto:+.4f} ({diferenca_direto*100:+.1f}%)")
    print(f"   vs Baseline Teórico: {diferenca_teorico:+.4f} ({diferenca_teorico*100:+.1f}%)")
    
    # Análise adicional
    print(f"\n🔬 ANÁLISE DETALHADA:")
    print(f"   • Acurácia por etapa:")
    print(f"     - Espécie: {metricas['acc_especie']:.4f} ({metricas['acc_especie']*100:.1f}%)")
    print(f"     - Saúde: {metricas['acc_saude']:.4f} ({metricas['acc_saude']*100:.1f}%)")
    print(f"     - Completo: {metricas['acc_combined']:.4f} ({metricas['acc_combined']*100:.1f}%)")
    
    print(f"   • Confiança média:")
    print(f"     - Espécie: {metricas['conf_especie_media']:.3f}")
    print(f"     - Saúde: {metricas['conf_saude_media']:.3f}")
    print(f"     - Final: {metricas['conf_final_media']:.3f}")
    
    print(f"\n🎯 CONCLUSÕES:")
    if diferenca_direto > 0:
        print(f"   ✅ Pipeline hierárquico superou o baseline direto em {abs(diferenca_direto)*100:.1f}%")
    else:
        print(f"   ❌ Pipeline hierárquico ficou {abs(diferenca_direto)*100:.1f}% abaixo do baseline direto")
    
    if abs(diferenca_teorico) < 0.01:
        print(f"   ⚖️ Pipeline hierárquico está muito próximo do baseline teórico (diferença: {diferenca_teorico*100:.1f}%)")
    elif diferenca_teorico > 0:
        print(f"   ✅ Pipeline hierárquico superou o baseline teórico em {abs(diferenca_teorico)*100:.1f}%")
    else:
        print(f"   ❌ Pipeline hierárquico ficou {abs(diferenca_teorico)*100:.1f}% abaixo do baseline teórico")
    
    return {
        'acc_hierarquico': acc_hierarquico,
        'acc_baseline_direto': acc_baseline_direto,
        'acc_baseline_teorico': acc_baseline_teorico,
        'diferenca_direto': diferenca_direto,
        'diferenca_teorico': diferenca_teorico,
    }

# Executar comparação final
comparacao = comparacao_final(metricas, acc_baseline_direto, acc_baseline_teorico)



🔍 Calculando baselines para comparação...
📊 BASELINE DIRETO (Classificação baseada apenas em espécie):
   Método: Predição de espécie + classificação majoritária de saúde
   Tomato: classe majoritária = unhealthy
   Potato: classe majoritária = unhealthy
   Pepper_bell: classe majoritária = healthy
   Acertos baseline: 316/450
   Acurácia baseline: 0.7022 (70.2%)

📊 BASELINE TEÓRICO (Multiplicação de acurácias independentes):
   Acurácia espécie: 0.8867 (88.7%)
   Acurácia saúde: 0.8667 (86.7%)
   Acurácia teórica: 0.7684 (76.8%)

📊 RESULTADOS FINAIS:
   🔗 Pipeline Hierárquico: 0.7689 (76.9%)
   📍 Baseline Direto: 0.7022 (70.2%)
   📐 Baseline Teórico: 0.7684 (76.8%)

📈 COMPARAÇÕES:
   vs Baseline Direto: +0.0667 (+6.7%)
   vs Baseline Teórico: +0.0004 (+0.0%)

🔬 ANÁLISE DETALHADA:
   • Acurácia por etapa:
     - Espécie: 0.8867 (88.7%)
     - Saúde: 0.8667 (86.7%)
     - Completo: 0.7689 (76.9%)
   • Confiança média:
     - Espécie: 0.867
     - Saúde: 0.845
     - Final: 0.731

🎯 CONC

In [7]:
# 8. ANÁLISE ADICIONAL E INTERPRETAÇÃO DOS RESULTADOS

print("📚 INTERPRETAÇÃO DOS BASELINES:")
print("=" * 60)

print("\n1️⃣ BASELINE DIRETO (Espécie + Classe Majoritária):")
print("   • Usa a mesma predição de espécie do pipeline hierárquico")
print("   • Para saúde, sempre escolhe a classe majoritária (mais comum)")
print("   • Representa um classificador simples que 'chuta' a saúde")
print("   • Mais realista para comparação com pipeline hierárquico")

print("\n2️⃣ BASELINE TEÓRICO (Multiplicação de Acurácias):")
print("   • Multiplica: Acurácia_Espécie × Acurácia_Saúde")
print("   • Assume que os erros são independentes")
print("   • Representa o limite teórico se não houvesse correlação")
print("   • Útil para detectar se hierarquia ajuda ou atrapalha")

print("\n🔍 ANÁLISE DA DISTRIBUIÇÃO DE CLASSES:")
print("   Distribuição de saúde por espécie (dados reais):")

for especie in df_resultados['especie_real'].unique():
    subset = df_resultados[df_resultados['especie_real'] == especie]
    healthy_count = len(subset[subset['saude_real'] == 'healthy'])
    unhealthy_count = len(subset[subset['saude_real'] == 'unhealthy'])
    total = len(subset)
    
    print(f"   • {especie}: {healthy_count} healthy, {unhealthy_count} unhealthy")
    print(f"     Proporção unhealthy: {unhealthy_count/total:.1%}")

print("\n💡 INTERPRETAÇÃO DOS RESULTADOS:")
print("   • Se Pipeline ≈ Baseline Teórico → Hierarquia não interfere")
print("   • Se Pipeline > Baseline Teórico → Hierarquia está ajudando")
print("   • Se Pipeline < Baseline Teórico → Hierarquia está atrapalhando")
print("   • Se Pipeline > Baseline Direto → Classificação de saúde é útil")

# Análise de erros por combinação
print(f"\n🔬 ANÁLISE DE ERROS POR COMBINAÇÃO:")
erros_combinacao = df_resultados[df_resultados['acerto_combined'] == False]
print(f"   Total de erros: {len(erros_combinacao)}")

print("\n   Tipos de erro:")
erro_apenas_especie = len(erros_combinacao[erros_combinacao['acerto_especie'] == False])
erro_apenas_saude = len(erros_combinacao[
    (erros_combinacao['acerto_especie'] == True) & 
    (erros_combinacao['acerto_saude'] == False)
])
erro_ambos = len(erros_combinacao[
    (erros_combinacao['acerto_especie'] == False) & 
    (erros_combinacao['acerto_saude'] == False)
])

print(f"   • Erro apenas na espécie: {erro_apenas_especie}")
print(f"   • Erro apenas na saúde: {erro_apenas_saude}")
print(f"   • Erro em ambos: {erro_ambos}")

print(f"\n🎯 EFICIÊNCIA DO PIPELINE:")
acc_hierarquico = comparacao['acc_hierarquico']
acc_teorico = comparacao['acc_baseline_teorico']
acc_direto = comparacao['acc_baseline_direto']

eficiencia_teorica = (acc_hierarquico / acc_teorico) * 100
eficiencia_direta = (acc_hierarquico / acc_direto) * 100

print(f"   • Eficiência vs Teórico: {eficiencia_teorica:.1f}%")
print(f"   • Eficiência vs Direto: {eficiencia_direta:.1f}%")

if eficiencia_teorica > 95:
    print("   ✅ Pipeline hierárquico está funcionando muito bem!")
elif eficiencia_teorica > 85:
    print("   ✅ Pipeline hierárquico está funcionando bem")
else:
    print("   ⚠️ Pipeline hierárquico pode ser melhorado")


📚 INTERPRETAÇÃO DOS BASELINES:

1️⃣ BASELINE DIRETO (Espécie + Classe Majoritária):
   • Usa a mesma predição de espécie do pipeline hierárquico
   • Para saúde, sempre escolhe a classe majoritária (mais comum)
   • Representa um classificador simples que 'chuta' a saúde
   • Mais realista para comparação com pipeline hierárquico

2️⃣ BASELINE TEÓRICO (Multiplicação de Acurácias):
   • Multiplica: Acurácia_Espécie × Acurácia_Saúde
   • Assume que os erros são independentes
   • Representa o limite teórico se não houvesse correlação
   • Útil para detectar se hierarquia ajuda ou atrapalha

🔍 ANÁLISE DA DISTRIBUIÇÃO DE CLASSES:
   Distribuição de saúde por espécie (dados reais):
   • Tomato: 30 healthy, 270 unhealthy
     Proporção unhealthy: 90.0%
   • Potato: 30 healthy, 60 unhealthy
     Proporção unhealthy: 66.7%
   • Pepper_bell: 30 healthy, 30 unhealthy
     Proporção unhealthy: 50.0%

💡 INTERPRETAÇÃO DOS RESULTADOS:
   • Se Pipeline ≈ Baseline Teórico → Hierarquia não interfere
  

In [8]:
# 9. SALVAMENTO DO RELATÓRIO FINAL

import json
from datetime import datetime

# Preparar dados para salvamento
relatorio_final = {
    'timestamp': datetime.now().isoformat(),
    'metricas_pipeline': {
        'acuracia_especie': float(metricas['acc_especie']),
        'acuracia_saude': float(metricas['acc_saude']),
        'acuracia_completa': float(metricas['acc_combined']),
        'confianca_especie': float(metricas['conf_especie_media']),
        'confianca_saude': float(metricas['conf_saude_media']),
        'confianca_final': float(metricas['conf_final_media'])
    },
    'baselines': {
        'direto': float(comparacao['acc_baseline_direto']),
        'teorico': float(comparacao['acc_baseline_teorico'])
    },
    'comparacoes': {
        'diferenca_vs_direto': float(comparacao['diferenca_direto']),
        'diferenca_vs_teorico': float(comparacao['diferenca_teorico']),
        'eficiencia_teorica': float(eficiencia_teorica),
        'eficiencia_direta': float(eficiencia_direta)
    },
    'analise_erros': {
        'total_erros': int(len(erros_combinacao)),
        'erro_apenas_especie': int(erro_apenas_especie),
        'erro_apenas_saude': int(erro_apenas_saude),
        'erro_ambos': int(erro_ambos)
    },
    'total_amostras': int(len(df_resultados))
}

# Salvar relatório
with open('datasets_processados/relatorio_pipeline_hierarquico.json', 'w') as f:
    json.dump(relatorio_final, f, indent=2)

print("📝 RELATÓRIO FINAL SALVO:")
print("   • Arquivo: datasets_processados/relatorio_pipeline_hierarquico.json")
print("   • Resultados detalhados: datasets_processados/resultados_pipeline_hierarquico.csv")
print("   • Conjunto de teste: datasets_processados/conjunto_teste_hierarquico.csv")

print(f"\n🎯 RESUMO EXECUTIVO:")
print(f"   • Pipeline Hierárquico: {metricas['acc_combined']:.1%}")
print(f"   • Baseline Direto: {comparacao['acc_baseline_direto']:.1%}")
print(f"   • Baseline Teórico: {comparacao['acc_baseline_teorico']:.1%}")
print(f"   • Eficiência: {eficiencia_teorica:.1f}%")

print(f"\n✅ ANÁLISE COMPLETA DO PIPELINE HIERÁRQUICO CONCLUÍDA!")
print("   Todos os resultados foram salvos para análise posterior.")


📝 RELATÓRIO FINAL SALVO:
   • Arquivo: datasets_processados/relatorio_pipeline_hierarquico.json
   • Resultados detalhados: datasets_processados/resultados_pipeline_hierarquico.csv
   • Conjunto de teste: datasets_processados/conjunto_teste_hierarquico.csv

🎯 RESUMO EXECUTIVO:
   • Pipeline Hierárquico: 76.9%
   • Baseline Direto: 70.2%
   • Baseline Teórico: 76.8%
   • Eficiência: 100.1%

✅ ANÁLISE COMPLETA DO PIPELINE HIERÁRQUICO CONCLUÍDA!
   Todos os resultados foram salvos para análise posterior.
